
Association rule mining using Apriori and FP Growth algorithm.

In [5]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

In [3]:
df = pd.read_excel('Online Retail.xlsx')

In [4]:
df.head()

,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.shape

(541909, 9)

In [7]:
df.Description = df.Description.str.strip()
df.InvoiceNo = df.InvoiceNo.astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df.isnull().sum()

InvoiceNo           0
StockCode           0
lower          530817
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     134697
Country             0
dtype: int64

In [8]:
new_df = (df[df.Country == 'France']
          .groupby(['InvoiceNo', 'Description']).Quantity
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
# new_df

In [9]:
def encode_units(x):
    if x <= 0:
        return 0
    else: 
        return 1
new_data = new_df.applymap(encode_units)
new_data.drop('POSTAGE', inplace=True, axis=1) 
# new_data

In [10]:
# The support is computed as the fraction
#   `transactions_where_item(s)_occur / total_transactions`.

#ferquent itemset is an itemset whose support value is greater than a specified thresold value

# Minimal threshold for the evaluation metric, via the `metric` parameter,to decide whether a candidate rule is of interest.
frequent_itemset = apriori(new_data, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_itemset, metric='lift',min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061


In [11]:
rules[(rules.lift >= 6) & (rules.confidence >= 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
17,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
19,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
20,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
21,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
22,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850


In [16]:
freq_fp=fpgrowth(new_data, min_support=0.1, use_colnames=True, verbose=1)
freq_fp


22 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (RED TOADSTOOL LED NIGHT LIGHT)
0 itemset(s) from tree conditioned on items (ROUND SNACK BOXES SET OF4 WOODLAND)
0 itemset(s) from tree conditioned on items (SPACEBOY LUNCH BOX)
0 itemset(s) from tree conditioned on items (MINI PAINT SET VINTAGE)
0 itemset(s) from tree conditioned on items (ALARM CLOCK BAKELIKE PINK)
0 itemset(s) from tree conditioned on items (LUNCH BAG RED RETROSPOT)
0 itemset(s) from tree conditioned on items (LUNCH BOX WITH CUTLERY RETROSPOT)
0 itemset(s) from tree conditioned on items (RED RETROSPOT MINI CASES)
0 itemset(s) from tree conditioned on items (LUNCH BAG WOODLAND)
0 itemset(s) from tree conditioned on items (PLASTERS IN TIN WOODLAND ANIMALS)
1 itemset(s) from tree conditioned on items (PLASTERS IN TIN SPACEBOY)
0 itemset(s) from tree conditioned on items (REGENCY CAKESTAND 3 TIER)
0 itemset(s) from tree conditioned on items (STRAWBERRY LUNCH BOX WITH CUTLERY)
0 i

,support,itemsets
0,0.181122,(RED TOADSTOOL LED NIGHT LIGHT)
1,0.158163,(ROUND SNACK BOXES SET OF4 WOODLAND)
2,0.125000,(SPACEBOY LUNCH BOX)
3,0.104592,(MINI PAINT SET VINTAGE)
4,0.102041,(ALARM CLOCK BAKELIKE PINK)
5,0.153061,(LUNCH BAG RED RETROSPOT)
6,0.142857,(LUNCH BOX WITH CUTLERY RETROSPOT)
7,0.137755,(RED RETROSPOT MINI CASES)
8,0.117347,(LUNCH BAG WOODLAND)
9,0.170918,(PLASTERS IN TIN WOODLAND ANIMALS)


In [23]:
rules_fp=association_rules(freq_fp,metric="confidence",min_threshold=0.8)
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
1,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
2,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
